## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

In [3]:
# 2. Minimum and maximum temperature inputs
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter to new DataFrame based on prompt
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Salalah,OM,2021-10-28 23:14:07,17.0151,54.0924,77.09,78,20,4.61,few clouds
7,7,Canyon Lake,US,2021-10-28 23:14:09,29.8752,-98.2625,76.05,28,0,4.00,clear sky
10,10,Kapaa,US,2021-10-28 23:13:41,22.0752,-159.3190,79.48,82,20,4.00,light rain
16,16,Butaritari,KI,2021-10-28 23:14:12,3.0707,172.7902,82.20,69,8,12.03,clear sky
19,19,Manaus,BR,2021-10-28 23:14:12,-3.1019,-60.0250,81.10,89,20,2.30,few clouds
27,27,Uvalde,US,2021-10-28 23:14:15,29.3669,-99.7670,79.14,16,1,11.50,clear sky
35,35,Bowen,AU,2021-10-28 23:14:17,-20.0167,148.2333,80.67,61,4,7.81,clear sky
37,37,Noumea,NC,2021-10-28 23:14:17,-22.2763,166.4572,82.38,99,1,12.33,clear sky
41,41,Tambacounda,SN,2021-10-28 23:14:19,13.7539,-13.7586,77.18,77,100,8.39,overcast clouds
42,42,Qandala,SO,2021-10-28 23:14:19,11.4720,49.8728,79.70,53,1,4.83,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Salalah,OM,77.09,few clouds,17.0151,54.0924,
7,Canyon Lake,US,76.05,clear sky,29.8752,-98.2625,
10,Kapaa,US,79.48,light rain,22.0752,-159.3190,
16,Butaritari,KI,82.20,clear sky,3.0707,172.7902,
19,Manaus,BR,81.10,few clouds,-3.1019,-60.0250,
27,Uvalde,US,79.14,clear sky,29.3669,-99.7670,
35,Bowen,AU,80.67,clear sky,-20.0167,148.2333,
37,Noumea,NC,82.38,clear sky,-22.2763,166.4572,
41,Tambacounda,SN,77.18,overcast clouds,13.7539,-13.7586,
42,Qandala,SO,79.70,clear sky,11.4720,49.8728,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found")
        

Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('',np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Description}</dd>
<dt>Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))